In [1]:
!pip install torchvision

In [2]:
import torch
import torch.nn as nn
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
save_dir = "diffusion_training_data/"
#save_dir = '/content/drive/MyDrive/ai ai pong/'+ save_dir


In [4]:
from torchvision.models import resnet18
encoder = resnet18(weights="IMAGENET1K_V1")
# #remove the fully connected layers
encoder.fc = nn.Identity()

#test it
encoder.eval()
x = torch.randn(1, 3, 224, 224) # will need to convert pong images from 160 and 210 to this size of 224,224 later.
with torch.no_grad():
    y = encoder(x)
    print(y.shape)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/codespace/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100.0%


torch.Size([1, 512])


In [10]:
# !pip install torchvision
# !pip install tensorboard
# !pip uninstall conditional_diffusion

#install torch with cuda
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install git+https://github.com/stevenharperja/conditional_diffusion.git#egg=conditional_diffusion


Looking in indexes: https://download.pytorch.org/whl/cu118
  Cloning https://github.com/stevenharperja/conditional_diffusion.git to /tmp/pip-install-vfa114x_/conditional-diffusion_41d00cc9b85d42f49c998ef56c0a16cf
  Running command git clone --filter=blob:none --quiet https://github.com/stevenharperja/conditional_diffusion.git /tmp/pip-install-vfa114x_/conditional-diffusion_41d00cc9b85d42f49c998ef56c0a16cf
  Resolved https://github.com/stevenharperja/conditional_diffusion.git to commit ce480bac794a41df4a5e88f5d7e04dba876b7fc4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [6]:
torch.cuda.is_available()
import torch

In [11]:
#import modules #dont need it.
import conditional_diffusion.modules as modules
from conditional_diffusion.ddpm_conditional import Diffusion as Diffusion
import conditional_diffusion.utils as utils
import torch



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

n = 1

model = modules.UNet_conditional(num_classes=10).to(device)
ckpt = torch.load("models/conditional_ema_ckpt.pt", map_location=device)
model.load_state_dict(ckpt)
diffusion = Diffusion(img_size=64, device=device)
print(model)
#Y is embedding
y = torch.Tensor([6] * n).long().to(device)
#X is image
x = diffusion.sample(model, n, y, cfg_scale=3) #set cfg_scale to 0 for half the lag?
utils.plot_images(x)

09:42:30 - INFO: Sampling 1 new images....


UNet_conditional(
  (inc): DoubleConv(
    (double_conv): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): GroupNorm(1, 64, eps=1e-05, affine=True)
      (2): GELU(approximate='none')
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): GroupNorm(1, 64, eps=1e-05, affine=True)
    )
  )
  (down1): Down(
    (maxpool_conv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): DoubleConv(
        (double_conv): Sequential(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): GroupNorm(1, 64, eps=1e-05, affine=True)
          (2): GELU(approximate='none')
          (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (4): GroupNorm(1, 64, eps=1e-05, affine=True)
        )
      )
      (2): DoubleConv(
        (double_conv): Sequenti

301it [00:11, 27.29it/s]


KeyboardInterrupt: 

In [ ]:
## upscaler model from https://github.com/yjn870/SRCNN-pytorch/blob/master/models.py ##UNUSED
class SRCNN(nn.Module):
    def __init__(self, num_channels=1):
        super(SRCNN, self).__init__()
        self.conv1 = nn.Conv2d(num_channels, 64, kernel_size=9, padding=9 // 2)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=5, padding=5 // 2)
        self.conv3 = nn.Conv2d(32, num_channels, kernel_size=5, padding=5 // 2)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.conv3(x)
        return x

In [ ]:
upscaler_weights_file = "./models/srcnn_x4.pth"

In [48]:
sample_im = torch.ones(0, 1, 64, 64)#.cuda()
l1 = nn.ConvTranspose2d(in_channels=1, out_channels=1, kernel_size=2, stride=2, padding=8, bias=False)
l2 = nn.ConvTranspose2d(in_channels=1, out_channels=1, kernel_size=2, stride=2, padding=0, bias=False)
sample_deconv = nn.Sequential(
                l1,
                nn.ReLU(),
                l2,
                nn.ReLU(),
)
l1.weight = torch.nn.Parameter(
    torch.tensor([[[[ 1.,  1.], 
                    [ 2.,  3.], 
                    ]]]))#.cuda())
l2.weight = torch.nn.Parameter(
    torch.tensor([[[[ 1.,  1.], 
                    [ 2.,  3.], 
                    ]]]))#.cuda())
print(sample_deconv(sample_im))

alternate_upscaler =  nn.Sequential(
                nn.ConvTranspose2d(in_channels=1, out_channels=1, kernel_size=2, stride=2, padding=8, bias=False),
                nn.ReLU(),
                nn.ConvTranspose2d(in_channels=1, out_channels=1, kernel_size=2, stride=2, padding=0, bias=False),
                nn.ReLU(),
)

tensor([], size=(0, 1, 224, 224), grad_fn=<ReluBackward0>)


In [12]:
!pip install tensorboard
import conditional_diffusion.modules as modules
from conditional_diffusion.ddpm_conditional import Diffusion as Diffusion
import conditional_diffusion.utils as utils
import torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 58.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 75.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 81.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.3/227.3 kB 9.3 MB/s eta 0:00:00


In [1]:
from tqdm import tqdm 
class Net(nn.Module):
    def __init__(self, device, embedding_scale = 0):
        super(Net, self).__init__()
        self.embedding_scale = embedding_scale

        resnet = resnet18(weights="IMAGENET1K_V1")
        resnet.fc = nn.Identity()
        #freeze the weights for resnet
        for param in resnet.parameters():
            param.requires_grad = False

        ###init and modify diffusion model
        diffusion_model = modules.UNet_conditional(num_classes=10).to(device)
        ckpt = torch.load("models/conditional_ema_ckpt.pt", map_location=device)
        diffusion_model.load_state_dict(ckpt)
        #remove nn.embedder and just send an embedding instead
        diffusion_model.label_emb = nn.Identity()
        # #freeze the weights for diffusion model
        # for param in diffusion_model.parameters():
        #     param.requires_grad = False
        # #unfreeze all of the diffusion model's decoder layers
        # diffusion_model.up1.requires_grad = True
        # diffusion_model.sa4.requires_grad = True
        # diffusion_model.up2.requires_grad = True
        # diffusion_model.sa5.requires_grad = True
        # diffusion_model.up3.requires_grad = True
        # diffusion_model.sa6.requires_grad = True
        # diffusion_model.outc.requires_grad = True

        ###init upscaler model
        upscaler = nn.Sequential(
            nn.Upsample(size=(224,224), mode="bilinear", align_corners=True),
            modules.DoubleConv(in_channels=3,out_channels=3,residual=True),
            modules.DoubleConv(in_channels=3,out_channels=3,residual=True),
        )

        #input of (N, 3, 224, 224), output of (N, 256)
        self.encoder = nn.Sequential(
            resnet,
            nn.MaxPool1d(kernel_size=2, stride=2, padding=0), #Using a layer which isnt trainable so I can reduce the size while keeping it frozen.
        )
        
        self.diffusion_model = diffusion_model #input of (N,256) output of (N,3,64,64)
        self.upscaler = upscaler
        self.final_layer = nn.Conv2d(in_channels=3, out_channels=1, kernel_size=1, stride=1, padding=0), # reduce channel size to 1 for black and white
        #input of (N, 256), output of (N, 1)
        self.reward_maker = nn.Sequential(
            nn.Linear(in_features=256, out_features=1),
        )
        #input of (N, 256), output of (N, 1)
        self.done_maker = nn.Sequential(
            nn.Linear(in_features=256, out_features=1),
        )


        ###variables for the diffusion model
        self.noise_steps = 1000
        self.beta_start = 1e-4
        self.beta_end = 0.02
        self.beta = torch.linspace(self.beta_start, self.beta_end, self.noise_steps).to(device)
        self.alpha = 1. - self.beta
        self.alpha_hat = torch.cumprod(self.alpha, dim=0)
        self.device = device


    #prediction function. this may not work for training. but maybe since its pretrained it might? work
    def diffusion_sample(self, embedding): # see https://github.com/stevenharperja/conditional_diffusion/blob/main/conditional_diffusion/ddpm_conditional.py#L41
        n = embedding.size()[0]
        x = torch.randn((n, 3, 64, 64)).to(self.device)
        for i in tqdm(reversed(range(1, self.noise_steps)), position=0):
            t = (torch.ones(n) * i).long().to(self.device)
            predicted_noise = self.diffusion_model(x, t, embedding)
            if self.embedding_scale > 0:
                uncond_predicted_noise = self.diffusion_model(x, t, None)
                predicted_noise = torch.lerp(uncond_predicted_noise, predicted_noise, self.embedding_scale)
            alpha = self.alpha[t][:, None, None, None]
            alpha_hat = self.alpha_hat[t][:, None, None, None]
            beta = self.beta[t][:, None, None, None]
            if i > 1:
                noise = torch.randn_like(x)
            else:
                noise = torch.zeros_like(x)
            x = 1 / torch.sqrt(alpha) * (x - ((1 - alpha) / (torch.sqrt(1 - alpha_hat))) * predicted_noise) + torch.sqrt(beta) * noise
        return x


    def forward(self, x, t = None, noised_truth = None, use_embedding = True):
        """
        
        x: The input image (n,3,224,224)
        t: The timesteps vector (n)
        noised_truth: a noised version of the target image, used for training (n,3,64,64)
        use_embedding: Whether to use the embedding or not, useful for training faster?

        
        returns:
        if training:
            it returns a predicted noise, and an upscaled predicted noise, along with a predicted reward and done
        if not training
            it returns a predicted image, along with a reward and done.
        
        """

        embedding = None
        if (not self.training) or use_embedding:
            embedding = self.encoder(x) #(n,256)        
        unscaled_image = None
        if self.training:
            if noised_truth == None:
                raise Exception("Cannot train without a provided noised target image")
            if t == None:
                raise Exception("Cannot train without a provided timesteps vector")
            unscaled_image = self.diffusion_forward_train(noised_truth,t,embedding) #(n,3,64,64)
        else:
            unscaled_image = self.diffusion_sample(embedding) #(n,3,64,64) 

        image = self.upscaler(unscaled_image)
        image = self.final_layer(image)#(n,1,224,224)
        #if not self.train:
        image = (image.clamp(-1, 1) + 1) / 2
        image = (image * 255).type(torch.uint8)
        rew = self.done_maker(embedding) #(n,1)
        don = self.reward_maker(embedding) #(n,1)
        if self.training:
            return unscaled_image,image,rew,don
        else:
            return image,rew,don


ModuleNotFoundError: No module named 'tqdm'

In [14]:
# Create an instance of the network
net = Net(device).to(device)

In [16]:
from torch.utils.data import Dataset
# from torchvision import datasets
# from torchvision.transforms import ToTensor
from os import listdir
from os.path import isfile, join

class PongDataset(Dataset):
    def __init__(self, dir, device):
        self.dir = dir
        self.device = device
    def __len__(self):
        existing_files = [f for f in listdir(self.dir) if isfile(join(self.dir, f))]
        if existing_files:
            #This grabs the largest integer out of all the filenames (filter the string for digit chars, convert those chars to an int)
            i = max(*[int(''.join([i for i in f if i.isdigit()])) for f in existing_files])
            return i
        else:
            return 0
    def __getitem__(self, index):
        # transitions = np.load(save_dir + "transitions{i}.npz".format(index))
        # observations = transitions["observations"]
        # actions = transitions["actions"]
        # rewards = transitions["rewards"]
        # dones = transitions["dones"]

        #use data which was preformatted for the 'trivial' model
        input = torch.load(self.dir+"input{i}.pt".format(i=index)).to(self.device)
        truth = torch.load(self.dir+"truth{i}.pt".format(i=index))
        truth = tuple([t.to(self.device)for t in truth])

        return input, truth 

In [17]:
batch_size = 4
trainset = PongDataset(save_dir,device)
if __name__ == '__main__':
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=False)#, num_workers=1)

In [15]:
#see https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

# define loss function and optimizer
import torch.optim as optim

small_img_criterion = nn.MSELoss()
img_criterion = nn.MSELoss()
rew_criterion = nn.MSELoss()
don_criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)


image_importance = 10 #hyperparameter for weighting how important the image is in the loss function.


In [18]:
from conditional_diffusion.ddpm_conditional import Diffusion
from conditional_diffusion.ddpm_conditional import SummaryWriter
from conditional_diffusion.ddpm_conditional import plot_images
from conditional_diffusion.ddpm_conditional import save_images  
from conditional_diffusion.modules import EMA
import os
import logging

diffusion = Diffusion(img_size=224, device=device)
logger = SummaryWriter(os.path.join("runs", args.run_name))
ema = EMA(0.995)
ema_model = copy.deepcopy(model).eval().requires_grad_(False)

net.train()
for epoch in range(300):  # loop over the dataset multiple times

    logging.info(f"Starting epoch {epoch}:")
    pbar = tqdm(trainloader)


    for i, data in enumerate(pbar, 0):
        print(i)
        input, truth = data
        images = images.to(device)
        t = diffusion.sample_timesteps(images.shape[0]).to(device)
        x_t, noise = diffusion.noise_images(images, t)
        if np.random.random() < 0.1:
            labels = None
        predicted_noise = model(x_t, t, labels)


        #TODO copy code from conditional diffusion training cycle.
        #TODO ^ and for above, change the random noise to not be reallocated in memory so that the time complexity is less.

        # forward + backward + optimize
        small_img,img,r,d = net(input)
        loss0 = small_img_criterion(small_img,truth[0])
        loss1 = img_criterion(img,truth[1])
        loss2 = rew_criterion(r,truth[2])
        loss3 = don_criterion(d,truth[3])
        loss = loss0 + loss1 + loss2 + loss3

        # zero the parameter gradients
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        ema.step_ema(ema_model, model)

        pbar.set_postfix(MSE=loss.item())
        logger.add_scalar("MSE", loss.item(), global_step=epoch * l + i)

    if epoch % 10 == 0:
        labels = torch.arange(10).long().to(device)
        sampled_images = diffusion.sample(model, n=len(labels), labels=labels)
        ema_sampled_images = diffusion.sample(ema_model, n=len(labels), labels=labels)
        plot_images(sampled_images)
        save_images(sampled_images, os.path.join("results", args.run_name, f"{epoch}.jpg"))
        save_images(ema_sampled_images, os.path.join("results", args.run_name, f"{epoch}_ema.jpg"))
        torch.save(model.state_dict(), os.path.join("models", args.run_name, f"ckpt.pt"))
        torch.save(ema_model.state_dict(), os.path.join("models", args.run_name, f"ema_ckpt.pt"))
        torch.save(optimizer.state_dict(), os.path.join("models", args.run_name, f"optim.pt"))

print('Finished Training')

0


0it [00:00, ?it/s]


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [24]:
PATH = './models/pong_gen.pth'
torch.save(net.state_dict(), PATH)

In [25]:
net = Net(hidden_channels=hidden_channels)
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [89]:
input = None
net.eval()
for i, data in enumerate(trainloader, 0):
    print(i)
    input, truth = data
    break

output = net(input[0].unsqueeze(0))[0]
print(output.shape)

0


999it [00:20, 48.82it/s]

torch.Size([1, 1, 224, 224])


In [90]:
from PIL import Image
image = Image.fromarray(input[0][0].cpu().detach().numpy())# true value
image.show()

image = Image.fromarray(output[0][0].cpu().detach().numpy())# predicted value
image.show()
print(output[0][0].cpu().detach().numpy())

[[  0 255 255 ... 194 255   0]
 [255 144   0 ... 194   0 164]
 [  0   0   0 ...   0   0 255]
 ...
 [194 194   0 ... 121   0 151]
 [  0   0   0 ...   0   0 255]
 [  0 255 255 ... 255 255   0]]


In [ ]:
gif = []
images = [Image.fromarray(observation) for observation in observations]
for image in images:
    gif.append(image)
gif[0].save('temp/result.gif', save_all=True,optimize=False, append_images=gif[1:], loop=0)

NameError: name 'observations' is not defined